In [11]:
import pandas as pd
import seaborn as sns
import os
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.misc import electrocardiogram
from scipy import signal
import scipy.integrate as integrate
import scipy.fftpack as ff
#!/usr/bin/env python

import numpy as np, os, sys, joblib
from scipy.io import loadmat
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from get_12ECG_features import get_12ECG_features

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from pathlib import Path
from numpy import genfromtxt

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv1D, LSTM, Dense, Dropout, TimeDistributed, Flatten
from keras.optimizers import Adam
import keras

import matplotlib.pyplot as plt
%matplotlib inline

# 1. Signal lecture 
#### Please note: 
For easier data analysis and features visualization, please use pandas (since its easier to use). But for the final signal (the one we need to integration), please use numpy format (convert to numpy using "signal.values")

In [13]:
features = f'features.csv'
labels = f'labels.csv'
testing = f'testing.csv'
testing = genfromtxt(testing, delimiter=',')
features = genfromtxt(features, delimiter=',')
labels = genfromtxt(labels, delimiter=',')

In [15]:
# Normalize data
testing = tf.keras.utils.normalize(testing)
features = tf.keras.utils.normalize(features)

In [17]:
print(features.dtype)
print(testing.dtype)
print(label.dtype)

float64
float64
float64


In [19]:
## Inside the run_12ECG_code we must consider the following structure (1,14)
testing_reshape = testing.reshape(1, -1)
imputer=SimpleImputer().fit(testing_reshape)
testing_reshape = imputer.transform(testing_reshape)
print(testing_reshape.shape)

(1, 14)


In [21]:
# Training set bust consider the next structure
print(features.shape, labels.shape)
4812*14

(4813, 14) (4813, 9)


67368

In [23]:
# Since labels are organized in hot-encoding format and we need the index Number, define decode:
def decode(datum):
    return np.argmax(datum)
label_final = []
    
# Extract hot-encode format to normal format
for i in range(labels.shape[0]):
    label = labels[i]
    decoded_labels = decode(label)
    label_final.append(decoded_labels)
# Convert list to np array
label_final = np.array(label_final)
label_final = np.uint8(label_final)
print(label_final.dtype)

uint8


#### Model 1 : branch NN_1

In [24]:
print("training model NN_1")
############# MODEL 1 - NN ######################
model_1 = tf.keras.models.Sequential()
model_1.add(tf.keras.layers.Flatten())
# use 128 neurons & use relu act func
model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))
model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))

model_1.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))
# the final output layer must be Dense and must fit # classifications and must use probability distribution instead of an activation function
model_1.add(tf.keras.layers.Dense(9, activation = tf.nn.softmax))
    
# Compile 
model_1.compile(
optimizer= 'adam' ,# NN intends to minimize losss, not maximize accuracy
loss = 'sparse_categorical_crossentropy',
metrics = ['accuracy']
)
# Fit the model appropiatelly
model_1.fit(features, label_final, epochs=10)
val_loss, val_acc = model_1.evaluate(features, label_final)
print("The model is trained")

training model NN_1
Epoch 1/10
151/151 [==============================] - 0s 2ms/step - loss: 2.0685 - accuracy: 0.2182
Epoch 2/10
151/151 [==============================] - 0s 1ms/step - loss: 2.0555 - accuracy: 0.2256
Epoch 3/10
151/151 [==============================] - 0s 1ms/step - loss: 2.0532 - accuracy: 0.2256
Epoch 4/10
151/151 [==============================] - 0s 2ms/step - loss: 2.0553 - accuracy: 0.2256
Epoch 5/10
151/151 [==============================] - 0s 1ms/step - loss: 2.0544 - accuracy: 0.2256
Epoch 6/10
151/151 [==============================] - 0s 1ms/step - loss: 2.0522 - accuracy: 0.2256
Epoch 7/10
151/151 [==============================] - 0s 2ms/step - loss: 2.0533 - accuracy: 0.2256
Epoch 8/10
151/151 [==============================] - 0s 1ms/step - loss: 2.0524 - accuracy: 0.2256
Epoch 9/10
151/151 [==============================] - 0s 1ms/step - loss: 2.0518 - accuracy: 0.2256
Epoch 10/10
151/151 [==============================] - 0s 739us/step - loss: 2.0

In [25]:
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 14)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               1920      
_________________________________________________________________
dense_8 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_9 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_10 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_11 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_12 (Dense)             (None, 128)              

In [26]:
# Save model.
model_1.save("NN_1.model")

INFO:tensorflow:Assets written to: NN_1.model\assets


In [27]:
print(features.shape, label_final.shape)

(4813, 14) (4813,)


In [29]:
nn_model = tf.keras.models.load_model('NN_1.model/')
nn_model

In [ ]:
### current label & score
From the baseline code we had that the original model saved three things:`model`, `imputer`, `classes` 

In [32]:
predictions = nn_model.predict([testing_reshape])[0]
predictions

array([0.09403204, 0.16651522, 0.02546168, 0.03028586, 0.09770199,
       0.08466987, 0.1342014 , 0.11385558, 0.25327644], dtype=float32)